### HW2

## Практическое задание

### 1. Попробуйте обучить нейронную сеть на Keras с другими параметрами. Опишите в комментарии к уроку - какой результата вы добились от нейросети? Что помогло вам улучшить ее точность?


In [67]:
# The full neural network code!
###############################
import numpy as np
import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical


In [68]:
import keras
import tensorflow as tf
from keras import backend as K
from keras.optimizers import SGD

config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 6} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)
K.tensorflow_backend._get_available_gpus()

[]

Cuda не установлена похоже. keras-gpu поставил. Либо она теано только ускоряет. хм. Было бы интересно использовать видюху. 

In [69]:

train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()

# Normalize the images.
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

# Flatten the images.
train_images = train_images.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))



In [70]:
# Build the model.
model = Sequential([
  Dense(64, activation='relu', input_shape=(784,)),
  Dense(128, activation='relu'),    
  Dense(128, activation='relu'),  
  Dense(10, activation='softmax'),
])

sgd = SGD(learning_rate=0.0099, momentum=0.9, nesterov=False, decay=1e-6)
# Compile the model.
model.compile(
  optimizer=sgd,
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

# Train the model.
model.fit(
  train_images,
  to_categorical(train_labels),
  epochs=6,
  batch_size=64,
)


# Evaluate the model.
model.evaluate(
  test_images,
  to_categorical(test_labels)
)

# Save the model to disk.
model.save_weights('model.h5')

# Load the model from disk later using:
# model.load_weights('model.h5')

# Predict on the first 5 test images.
predictions = model.predict(test_images[:5])

# Print our model's predictions.
print(np.argmax(predictions, axis=1)) # [7, 2, 1, 0, 4]

# Check our predictions against the ground truths.
print(test_labels[:5]) # [7, 2, 1, 0, 4]

Epoch 1/6
60000/60000 [==============================] - 2s 39us/step - loss: 0.4262 - accuracy: 0.8673
Epoch 2/6
60000/60000 [==============================] - 2s 37us/step - loss: 0.1877 - accuracy: 0.9419
Epoch 3/6
60000/60000 [==============================] - 2s 34us/step - loss: 0.1406 - accuracy: 0.9564
Epoch 4/6
60000/60000 [==============================] - ETA: 0s - loss: 0.1165 - accuracy: 0.96 - 2s 34us/step - loss: 0.1159 - accuracy: 0.9638
Epoch 5/6
60000/60000 [==============================] - 2s 36us/step - loss: 0.0971 - accuracy: 0.9691
Epoch 6/6
10000/10000 [==============================] - 1s 58us/step
[7 2 1 0 4]
[7 2 1 0 4]


#### Результат:
    SGD Optimizer уменьшил время обучения. 
    Другие функции активации результат не улучшили.
    ДОбавление еше одного скрытого слоя на качество практически не влияет. 
    Увеличение узлов в скрытом слое добавляет качество обучения.
    Увеличение batch-size уменьшает скорость обучения, но есть порог при котором ухудшается качество.
    Увеличение эпох влияет на точность, главное не переобучить модель. 

 ### 2. Поработайте с документацией Keras.


Можно остановиться, если ошибка перестала уменьшаться используя 
from keras.callbacks import EarlyStopping  

Удобно описывать модель, смешивая оба способа — Functional API и Sequential API, что бы извлекать, сохранять и применять к новым входным данным куски модели

Есть метод извлечение слоя.